<a href="https://colab.research.google.com/github/lorenafc/MGI/blob/main/AuthorSmallCitiesGeoref.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#installing libraries

!apt update -q
!apt upgrade -q
!apt install gdal-bin python-gdal python3-gdal -q
!apt install python3 -rtree -q
!pip install geopandas==0.10.0 -q

!pip install geopandas geopy -q

!pip install descartes -q

# Install unidecode to fix encoding issue
!pip install unidecode -q
!pip install openpyxl -q

from unidecode import unidecode

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [53.5 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [917 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,439 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,160 kB]
Get:1

In [3]:
#import libraries
import pandas as pd
import geopandas as gpd

In [6]:
from geopandas.tools import geocode

In [8]:
# upload the file from your computer
from google.colab import files
uploaded=files.upload()

Saving additional_authors.xlsx to additional_authors.xlsx


In [16]:
# Assuming the uploaded file is named 'additional_authors.xlsx'
file_name = 'additional_authors.xlsx'

# Read the Excel file
authors_small_cities = pd.read_excel(file_name, engine='openpyxl')

authors_small_cities_copy = authors_small_cities

In [32]:
# # Function to fix encoding issues #source chatgpt
### THIS FUNCTION SHOWS THE SPECIAL CARACHTERS, HOWEVER NOT ALL OF THEM. there are still "?"



def fix_encoding(text):
    if isinstance(text, str):
        try:
            # Try decoding the text as UTF-8
            decoded_text = text.encode('latin1', errors='replace').decode('utf-8', errors='replace')
            return decoded_text
        except UnicodeDecodeError:
            try:
                # If decoding as UTF-8 fails, try encoding as latin1
                encoded_text = text.encode('latin1', errors='replace').decode('latin1', errors='replace')
                return encoded_text
            except UnicodeEncodeError:
                # If encoding as latin1 fails, return the original text with unencodable characters replaced
                return text.encode('latin1', errors='replace').decode('latin1', errors='replace')
    return text

## tried this one with more options, but didint work


# def fix_encoding(text):
#     if isinstance(text, str):
#         try:
#             # Try decoding the text as if it was misinterpreted as Latin-1 and then encoded as UTF-8
#             decoded_text = text.encode('latin1', errors='replace').decode('utf-8', errors='replace')
#             return decoded_text
#         except UnicodeDecodeError:
#             pass

#         try:
#             # Try decoding the text as if it was misinterpreted as Windows-1252 and then encoded as UTF-8
#             decoded_text = text.encode('windows-1252', errors='replace').decode('utf-8', errors='replace')
#             return decoded_text
#         except UnicodeDecodeError:
#             pass

#         try:
#             # Try decoding the text as Latin-1
#             decoded_text = text.encode('latin1', errors='replace').decode('latin1', errors='replace')
#             return decoded_text
#         except UnicodeDecodeError:
#             pass

#         try:
#             # Try decoding the text as Windows-1252
#             decoded_text = text.encode('windows-1252', errors='replace').decode('windows-1252', errors='replace')
#             return decoded_text
#         except UnicodeDecodeError:
#             pass

#         try:
#             # Handle double encoding scenario: UTF-8 encoded text misinterpreted as Latin-1, then Latin-1 encoded text misinterpreted as UTF-8
#             first_pass = text.encode('utf-8', errors='replace').decode('latin1', errors='replace')
#             decoded_text = first_pass.encode('latin1', errors='replace').decode('utf-8', errors='replace')
#             return decoded_text
#         except (UnicodeEncodeError, UnicodeDecodeError):
#             pass
#         try:
#             # Handle double encoding scenario: windows-1252 encoded text misinterpreted as Latin-1, then Latin-1 encoded text misinterpreted as windows1252
#             first_pass = text.encode('windows-1252', errors='replace').decode('latin1', errors='replace')
#             decoded_text = first_pass.encode('latin1', errors='replace').decode('windows-1252', errors='replace')
#             return decoded_text
#         except (UnicodeEncodeError, UnicodeDecodeError):
#             pass
#         try:
#             # Handle double encoding scenario: latin1 encoded text misinterpreted as win1252, then win1252 encoded text misinterpreted as latin1
#             first_pass = text.encode('latin1', errors='replace').decode('windows-1252', errors='replace')
#             decoded_text = first_pass.encode('windows-1252', errors='replace').decode('latin1', errors='replace')
#             return decoded_text
#         except (UnicodeEncodeError, UnicodeDecodeError):
#             pass
#         try:
#             # Handle double encoding scenario: latin2 encoded text misinterpreted as utf-8, then utf-8 encoded text misinterpreted as latin2
#             first_pass = text.encode('latin2', errors='replace').decode('utf-8', errors='replace')
#             decoded_text = first_pass.encode('utf-8', errors='replace').decode('latin2', errors='replace')
#             return decoded_text
#         except (UnicodeEncodeError, UnicodeDecodeError):
#             pass
#         try:
#             # Handle double encoding scenario: utf-8 encoded text misinterpreted as latin2, then latin2 encoded text misinterpreted as utf8
#             first_pass = text.encode('uft-8', errors='replace').decode('latin2', errors='replace')
#             decoded_text = first_pass.encode('latin2', errors='replace').decode('utf-8', errors='replace')
#             return decoded_text
#         except (UnicodeEncodeError, UnicodeDecodeError):
#             pass
#         try:
#             # Handle double encoding scenario: UTF-8 encoded text misinterpreted as Windows-1252, then Windows-1252 encoded text misinterpreted as UTF-8
#             first_pass = text.encode('utf-8', errors='replace').decode('windows-1252', errors='replace')
#             decoded_text = first_pass.encode('windows-1252', errors='replace').decode('utf-8', errors='replace')
#             return decoded_text
#         except (UnicodeEncodeError, UnicodeDecodeError):
#             pass

#     return text


# Apply the encoding fix function to the columns containing city names
authors_small_cities_copy['borncity'] = authors_small_cities_copy['born'].apply(fix_encoding)
authors_small_cities_copy['deathcity'] = authors_small_cities_copy['death'].apply(fix_encoding)
authors_small_cities_copy['activecity'] = authors_small_cities_copy['active'].apply(fix_encoding)


# Display the first few rows to verify the changes
print(authors_small_cities_copy[["born",'borncity', "death", 'deathcity']].head(50))



                              born                       borncity  \
0                                                                   
1                                                                   
2                            (Chur                          (Chur   
3                         A Capela                       A Capela   
4                        A GudiÃ±a                       A Gudiña   
5            Å ariÅ¡skÃ© Sokolovce             Šarišské Sokolovce   
6                         Å ibenik                        Šibenik   
7                         Å ibenik                        Šibenik   
8                         Å ibenik                        Šibenik   
9                      Å id Serbia                     Šid Serbia   
10                       Å ivetice                       Šivetice   
11                    Å kofja Loka                    Škofja Loka   
12      Å kvorci u ÄŒeskÃ©ho Brodu       Škvorci u �?eského Brodu   
13                        Å luknov

In [25]:
#renaming the columns of the copy
authors_small_cities_rename = authors_small_cities_copy.rename(columns = {'cleanedbirth':'bitrhyear',
                       'cleaneddeath':'deathyear', "finalname":"nameandbirthdeathyear"},
            inplace = True)


                         starturl  originalbirth  originaldeath bitrhyear  \
0   http://viaf.org/viaf/81048146         1570.0         1634.0      1570   
1  http://viaf.org/viaf/180581759            0.0         1775.0       NaN   
2  http://viaf.org/viaf/100161519            0.0         1570.0       NaN   
3  http://viaf.org/viaf/169350346         1699.0         1770.0      1699   
4   http://viaf.org/viaf/67541728         1502.0         1600.0      1502   

   deathyear                        nameandbirthdeathyear  \
0     1634.0             Carneiro, Tomé Tavares 1570-1634   
1     1775.0                     Kolíček, Caesarij, -1775   
2     1570.0                     Hertel, Jakob 15..?-1570   
3     1770.0  Ferro Caaveyro, Lucas Antonio ca. 1699-1770   
4     1600.0       Aparicio, Sebastián de 1502-1600 Santo   

                                     georeferenceurl        death       born  \
0                                  viafprocessedfile          NaN              
1         

In [26]:

authors_small_cities_cleaned = authors_small_cities_copy.drop(columns=["originalbirth","originaldeath"])


                         starturl bitrhyear  deathyear  \
0   http://viaf.org/viaf/81048146      1570     1634.0   
1  http://viaf.org/viaf/180581759       NaN     1775.0   
2  http://viaf.org/viaf/100161519       NaN     1570.0   
3  http://viaf.org/viaf/169350346      1699     1770.0   
4   http://viaf.org/viaf/67541728      1502     1600.0   

                         nameandbirthdeathyear  \
0             Carneiro, Tomé Tavares 1570-1634   
1                     Kolíček, Caesarij, -1775   
2                     Hertel, Jakob 15..?-1570   
3  Ferro Caaveyro, Lucas Antonio ca. 1699-1770   
4       Aparicio, Sebastián de 1502-1600 Santo   

                                     georeferenceurl        death       born  \
0                                  viafprocessedfile          NaN              
1                                  viafprocessedfile      Cieszin              
2     http://catalogue.bnf.fr/ark:/12148/cb15011774s          NaN      (Chur   
3  www.wikidata.org/wiki/Q3393

In [21]:
print(authors_small_cities_cleaned.shape)
col_names=list(authors_small_cities_cleaned.columns)
print(col_names)

(80802, 11)
['starturl', 'bitrhyear', 'deathyear', 'nameandbirthdeathyear', 'georeferenceurl', 'death', 'born', 'active', 'borncity', 'deathcity', 'activecity']


In [114]:
import numpy as np

In [27]:
#change the columns order:
authors_small_cities_cleaned=authors_small_cities_cleaned[['starturl', 'bitrhyear', 'deathyear', 'nameandbirthdeathyear', 'georeferenceurl', 'born', 'borncity', 'death', 'deathcity','active', 'activecity']]
#print(authors_small_cities_cleaned[["born", 'borncity', "death",'deathcity']].head(50))

                              born                       borncity  \
0                                                                   
1                                                                   
2                            (Chur                          (Chur   
3                         A Capela                       A Capela   
4                        A GudiÃ±a                       A Gudiña   
5            Å ariÅ¡skÃ© Sokolovce             Šarišské Sokolovce   
6                         Å ibenik                        Šibenik   
7                         Å ibenik                        Šibenik   
8                         Å ibenik                        Šibenik   
9                      Å id Serbia                     Šid Serbia   
10                       Å ivetice                       Šivetice   
11                    Å kofja Loka                    Škofja Loka   
12      Å kvorci u ÄŒeskÃ©ho Brodu       Škvorci u �?eského Brodu   
13                        Å luknov

In [38]:
# export the result as excel file
file_name = 'additonal_authors_cleaned_questionmark.xlsx'
authors_small_cities_cleaned.to_excel(file_name)

In [39]:
#ATTEMPT 1 GEOCODING

# Define the geocoding function source https://colab.research.google.com/drive/1Pi7Qv6fUW3NwhtDIiRSVyhS9qKMoXO4R?usp=sharing#scrollTo=fvWM7r6WGg_A
def geocode(address, api_key):
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {'address': address, 'key': api_key}
    response = requests.get(url, params=params)
    data = response.json()
    if data['status'] == 'OK':
        lat = data['results'][0]['geometry']['location']['lat']
        lng = data['results'][0]['geometry']['location']['lng']
        return lat, lng
    else:
        return None, None

# Add columns for latitude and longitude
authors_small_cities_cleaned['Latitude'] = None
authors_small_cities_cleaned['Longitude'] = None

# Google Maps Geocoding API key
api_key = 'YOUR_GOOGLE_API_KEY'  # Replace with your Google API key Its leading to a billing page

# Loop over the rows and geocode each address
for idx, row in df.iterrows():
    city = row['borncity']  # Replace 'place_name' with the name of the column containing the place names.
    lat, lng = geocode(city, api_key)
    authors_small_cities_cleaned.loc[idx, 'Latitude'] = lat
    authors_small_cities_cleaned.loc[idx, 'Longitude'] = lng

# Display the first few rows of the updated dataframe
authors_small_cities_cleaned.head()


KeyError: 'place_name'

In [36]:
## ATTEMPT 2  GEOCODING ###### not working

# Function to geocode the city names source: PyGEO course and chatgpt

def geocode_city(city_name):
    try:
        geo = geocode(city_name, provider="nominatim",  timeout=10)
        return geo.geometry.iloc[0] if not geo.empty else None
    except Exception as e:
        return None

# Apply the geocoding function to the 'borncity', 'deathcity', and 'activecity' columns
authors_small_cities_cleaned['borncity_geo'] = authors_small_cities_cleaned['borncity'].apply(geocode_city)
# authors_small_cities_cleaned['deathcity_geo'] = authors_small_cities_cleaned['deathcity'].apply(geocode_city)
# authors_small_cities_cleaned['activecity_geo'] = authors_small_cities_cleaned['activecity'].apply(geocode_city)

# Display the first few rows to verify the results
print(authors_small_cities_cleaned.head(10))

                         starturl bitrhyear  deathyear  \
0   http://viaf.org/viaf/81048146      1570     1634.0   
1  http://viaf.org/viaf/180581759       NaN     1775.0   
2  http://viaf.org/viaf/100161519       NaN     1570.0   
3  http://viaf.org/viaf/169350346      1699     1770.0   
4   http://viaf.org/viaf/67541728      1502     1600.0   
5  http://viaf.org/viaf/121452165      1320     1394.0   
6   http://viaf.org/viaf/62383957      1538     1592.0   
7   http://viaf.org/viaf/67541507      1530     1596.0   
8   http://viaf.org/viaf/78669554      1640     1705.0   
9   http://viaf.org/viaf/79445202      1731     1797.0   

                         nameandbirthdeathyear  \
0             Carneiro, Tomé Tavares 1570-1634   
1                     Kolíček, Caesarij, -1775   
2                     Hertel, Jakob 15..?-1570   
3  Ferro Caaveyro, Lucas Antonio ca. 1699-1770   
4       Aparicio, Sebastián de 1502-1600 Santo   
5                              Kukullei, Janos   
6          